In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd/content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 10


In [ ]:
# Load the labels from the Excel sheet
labels_df = pd.read_csv('./miml_dataset/miml_labels_2.csv')
labels_df.head()


,Filenames,labels
0,1.jpg,desert
1,2.jpg,desert
2,3.jpg,desert
3,4.jpg,"desert,mountains"
4,5.jpg,desert


In [ ]:
print(labels_df)


     Filenames            labels
0        1.jpg            desert
1        2.jpg            desert
2        3.jpg            desert
3        4.jpg  desert,mountains
4        5.jpg            desert
...        ...               ...
1995  1996.jpg             trees
1996  1997.jpg             trees
1997  1998.jpg             trees
1998  1999.jpg             trees
1999  2000.jpg             trees

[2000 rows x 2 columns]


In [ ]:
# Define the ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [ ]:
num_classes = len(labels_df['labels'].unique())

In [ ]:
train_set = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory='./miml_dataset/images',
    x_col='Filenames',
    y_col='labels',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42,
    validate_filenames=True,
    interpolation='bilinear',
    color_mode='rgb',
    save_format='jpg',
    num_classes=num_classes)

Found 1600 validated image filenames belonging to 20 classes.


In [ ]:
# Load the validation set
val_set = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory='./miml_dataset/images',
    x_col='Filenames',
    y_col='labels',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

Found 400 validated image filenames belonging to 20 classes.


In [ ]:
num_classes = len(labels_df['labels'].unique())


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

In [ ]:
# Define the InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='sigmoid')(x)

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
from tensorflow.keras.optimizers import Adam
# Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_set, validation_data=val_set, epochs=EPOCHS)

Epoch 1/10
50/50 [==============================] - 1067s 21s/step - loss: 1.3385 - accuracy: 0.5919 - val_loss: 13.1666 - val_accuracy: 0.0000e+00
Epoch 2/10
50/50 [==============================] - 1007s 20s/step - loss: 0.9462 - accuracy: 0.6712 - val_loss: 14.8400 - val_accuracy: 0.0000e+00
Epoch 3/10
50/50 [==============================] - 1008s 20s/step - loss: 0.8214 - accuracy: 0.7219 - val_loss: 20.0301 - val_accuracy: 0.0000e+00
Epoch 4/10
50/50 [==============================] - 1010s 20s/step - loss: 0.7996 - accuracy: 0.7375 - val_loss: 11.9887 - val_accuracy: 0.0000e+00
Epoch 5/10
50/50 [==============================] - 980s 20s/step - loss: 0.7333 - accuracy: 0.7500 - val_loss: 12.8299 - val_accuracy: 0.0000e+00
Epoch 6/10
50/50 [==============================] - 975s 20s/step - loss: 0.6959 - accuracy: 0.7681 - val_loss: 10.3715 - val_accuracy: 0.0000e+00
Epoch 7/10
50/50 [==============================] - 981s 20s/step - loss: 0.5703 - accuracy: 0.7962 - val_loss: 12